# Lab 3: Handling Missing Values In your Data

Machine learning models cannot work with missing values in a dataset (`NaNs` or `'-'`). It is crucial to fix any missing values in your data. The following are 2 ways to deal with missing data:
- deleting rows that contain missing features;
- replacing missing features using proper techniques.

In [1]:
#Importing library
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# loading the 'df_1_lab_3.csv' csv data
df = pd.read_csv('df_1_lab_3.csv')

In [3]:
# preparing data
df = df.drop(columns=['Unnamed: 0', 'index'])
df.head()

,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,2011-06-10,male,72,d,NaN,5.058993,1.481877,negative
1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
4,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive


# Lab Activity One: Handling Missing Data Without Time-Series Specific Methods

#### You will be dealing with `NaN` values for the ``Q`` and ``Mode`` columns. You will go through several ways of handling the missing values. For each method, you will first create a temporary copy of the master dataframe (`df`) to not alter the original dataset.

In [4]:
# Checking how many missing values are in the Q column
df["Q"].isnull().sum()

9

##### [A] Drop all the rows which contain `NaN` values in the `'Q'` column. Be sure to only drop the rows with missing values in the `'Q'` column and no other column. Remember to use the `df_temp` dataframe.

In [10]:
#making copy of the dataset
#filling all the missing values in column Q with 0
df_temp = df.copy()
df_temp['Q'] = df_temp['Q'].fillna(0)

##### [A] Using `df_temp` replace all the `NaN` values in the `'Q'` column with the *mean* of the `'Q'` column.

In [11]:
#making copy of the dataset
#filling all the missing values in column Q with mean of that column
df_temp = df.copy()
df_temp['Q'] = df_temp['Q'].fillna(df_temp['Q'].mean())

##### [A] Using `df_temp` replace all the `NaN` values in the `'Q'` column with the *median* of the `'Q'` column.

In [13]:
#making copy of the dataset
#filling all the missing values in column Q with median of that column
df_temp = df.copy()
#Like what you did for mean(), use median() instead.
df_temp['Q'] = df_temp['Q'].fillna(df['Q'].median())

##### [A] Using `df_temp` replace all the `NaN` values in the `'Q'` column with the *mode* of the `'Q'` column.
> Hint: Replace with the mode is trickier that the two previous methods. First see the unique values of column 'Q', then try the mode() fucntion and see the result.

In [14]:
#making the copy of the orginial dataset
df_temp = df.copy()
df_temp['Q'] = df_temp['Q'].fillna(df_temp['Q'].mode()[0], inplace = True)
#filling the nan values in the Q column with the mode of the particular column and selecting the first mode that we will get in case their are more than one mode

#### Now, you will be dealing with missing values for the `'Mode'` column. This contains categorical values, which are harder to replace with a logical value, unlike numeric (mean, median, mode). Below are the recommended steps for this:
- Replace with the most recurring category (similar to mode).
- Create a new category called 'missing' and replace with it that.
- Replace with a category depending on another feature (requires further data analysis).
- Simply delete the rows with missing values.

Note: For our dataset, instead of containing `NaN`s the `'Mode'` column has `'_'` to indicate missing values.

In [15]:
#making the copy of the orginial dataset
df_temp = df.copy()


##### [A] Using `df_temp`, replace the `'_'` values with the string `'missing'`.

In [21]:
#replacing the value '_' this with 'missing' in the column 'MODE'
df_temp['Mode']= df_temp['Q'].replace({'_':'missing'})

In [22]:
#replacing the value '_' this with 'missing' in the column 'MODE' using mode function and setting the first value to be the mode
df_temp = df.copy()
df_temp['Mode']= df_temp['Mode'].replace({'_':df_temp['Mode'].mode()[0]})

In [23]:
#inspecting the mode column
df_temp['Mode'].unique()

array(['d', 'e', 'f', 'g', 'h', 'a'], dtype=object)

# Lab Activity Two: Handling Missing Data With Time-Series Specific Methods

The three most popular methods for dealing with time-series data are listed below
- **Last Observation Carried Forward (LOCF):** Replacing the missing value with the value in the previous cell
- **Next Observation Carried Backward (NOCB):** Replacing the missing value with the value in the next cell
- **Linear interpolation:** Replacing missing values with estimates from previous values

##### Preparing the data

In [24]:
#Loading data
df_2 = pd.read_csv('df_2_lab_3.csv')

# Dropping unwated column
df_2 = df_2.drop(columns=['Unnamed: 0'])

# Change datatype of the date column
df_2["Inspection Date"] = pd.to_datetime(df_2["Inspection Date"])

# Here we are splitting our dataframe into 2, depend on if the site of device in is clinic 1 or 2
df_2_1 = df_2[df_2["Device Site"] == 'clinic1'].copy()
df_2_2 = df_2[df_2["Device Site"] == 'clinic2'].copy()
#reseting the index so that it will be in a order for both of the datasets that we make from the main one original dataset
df_2_1.reset_index(inplace=True, drop=True)
df_2_2.reset_index(inplace=True, drop=True)

In [25]:
#showing the first 5 rows of the data set
df_2_1.head()

,Inspection Date,R1,R3,Device Site
0,2010-09-04,0.079601,0.542921,clinic1
1,2010-09-27,NaN,9.613118,clinic1
2,2010-10-06,NaN,4.298943,clinic1
3,2010-10-21,0.877617,9.240019,clinic1
4,2010-11-26,NaN,6.659528,clinic1


In [26]:
#showing the first 5 rows of the data set
df_2_2.head()

,Inspection Date,R1,R3,Device Site
0,2011-01-04,0.927976,5.473454,clinic2
1,2011-01-28,0.905114,7.223218,clinic2
2,2011-02-04,0.683924,5.047370,clinic2
3,2011-02-27,0.622961,1.971804,clinic2
4,2011-04-10,NaN,10.822079,clinic2


##### Checking for missing values in the `'R1'` column

In [27]:
# Checking how many missing values are in the R1 column for both of the dataset that we make from the orginial dataet
print(df_2_1["R1"].isnull().sum())
print(df_2_2["R1"].isnull().sum())

4
2


##### [A] An important first step when using time-series specific methods is to sort the dataset by time (date in our case). Sort both `df_2_1` and `df_2_2` by the `'Inspection Date'` column.

In [28]:
#sorting the values of the inspection date column
df_2_1.sort_values(by="Inspection Date", inplace=True)
#Do the same thing for df_2_2

In [30]:
#sorting the inspection date column using sort function and sort it in ascending order
df_2_2.sort_values(by="Inspection Date", inplace=True)

#### Last Observation Carried Forward (LOCF)

##### [A] using `df_temp`, Use LOCF to fill the `NaN` values of the entire `df_2_1 dataset`.

> Hint: Use method `fillna` and set the `method` appropriately.

In [31]:
#making the copy of the original dataset
df_temp = df_2_1.copy()


In [32]:
#filling the NaN values for the entire dataset using method function
# Fill missing values in df_temp with the values from the previous row
df_temp = df_temp.fillna(method = 'ffill', inplace = True)


#### Next Observation Carried Backward (NOCB)



##### [A] Using `df_temp`, Use NOCB to fill the `NaN` values of the entire dataset.

 Hint: Use method `fillna` and set the `method` appropriately.

In [33]:
#making the copy of the original dataset
df_temp = df_2_2.copy()



In [34]:
#filling the NaN values for the entire dataset using method function
# Fill missing values in df_temp with the values from the forward row
df_temp = df_temp.fillna(method = 'bfill', inplace = True)


#### Linear interpolation



##### [A] Using `df_temp`, Use Interpolation to fill the NaN values of the `'R1'` column.
> Hint: Set the index of the dataframe to be the inspection column and use the `.interpolate function(method='index)`. Reading documentation of the functions will help you do this.

In [52]:
#making the copy of the original dataset
df_temp = df_2_1.copy()


In [53]:
#setting the inspection date column as the index for the dataset
df_temp.set_index('Inspection Date',inplace = True)
df_temp['R1'].interpolate(method='index',inplace = True)#using linear interpolation to fill the missing values

In [57]:
df_temp.head()

,R1,R3,Device Site
Inspection Date,,,
2010-09-04,0.079601,0.542921,clinic1
2010-09-27,0.470120,9.613118,clinic1
2010-10-06,0.622931,4.298943,clinic1
2010-10-21,0.877617,9.240019,clinic1
2010-11-26,0.761501,6.659528,clinic1


In [54]:
# Checking how many missing values are in the R1 column
df_temp['R1'].isnull().sum()

0